Version 1.0
---
A JSON agent that explores a greenery dataset. Has problems finding neighborhoods that are further down in the file because of an iteration limit.

Imports
---

In [2]:
from langchain_community.agent_toolkits.json.toolkit import JsonToolkit
from langchain_community.agent_toolkits.json.base import create_json_agent
from langchain_openai import AzureChatOpenAI
from langchain_community.tools.json.tool import JsonSpec
from langchain_ollama import ChatOllama
from dotenv import load_dotenv
import requests
import os
import json

Load a large language model
----------
Langchain makes it possible to easily switch LLMs. Llama 3 is used to show the data can be analysed with a locally running open-source model, but it is very slow (because it runs on a notebook). So to speed it up I also used o3-mini to show it works.

Load Llama3:


In [11]:
chosen_llm = ChatOllama(base_url='http://localhost:11434', model="llama3")

Load o3-mini (via Azure):

In [3]:
load_dotenv()

chosen_llm = AzureChatOpenAI(model ="o3-mini", api_version="2025-01-01-preview", azure_endpoint="https://56948-m9bdjgpg-eastus2.cognitiveservices.azure.com/openai/deployments/o3-mini/chat/completions?api-version=2025-01-01-preview", api_key=os.environ.get("AZURE_OPENAI_API_KEY"))

In [11]:
load_dotenv()

chosen_llm = AzureChatOpenAI(model="gpt-4o-mini", api_version="2025-01-01-preview",
                             azure_endpoint="https://56948-m9bdjgpg-eastus2.cognitiveservices.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview",
                             api_key=os.environ.get("AZURE_OPENAI_API_KEY"))

Retrieve dataset
---
Here the dataset is retrieved and stored in a file.

In [4]:
dataset = requests.get('https://data.rivm.nl/geo/ank/ows?service=WFS&request=GetFeature&typeName=rivm_2022_groenpercentage_kaart_per_buurt&propertyName=bu_naam,_mean&outputFormat=json').json()

# Print dataset to file to inspect it when needed
with open('groenPercentagePerBuurt.json', 'w') as file:
    json.dump(dataset, file)

Extract properties
---
Not used anymore, when sending only the properties the agent couldn't understand the context of the data anymore. This resulted in getting the response "I don't know" constantly.

In [ ]:
properties = []
neighborhoodGreenery = {}

for feature in dataset["features"]:
    properties.append(feature["properties"])

for property in properties:
    neighborhoodGreenery[property["bu_naam"]] = property["_mean"]

Create and run the agent
---
Here the toolkit is created that "gives" the dataset to the agent. It also creates the agent itself, what will analyse the dataset when it is executed.


In [20]:
# Create toolkit:
json_spec = JsonSpec(dict_=dataset, max_value_length=4000)
json_toolkit = JsonToolkit(spec=json_spec)

# Create JSON agent
json_agent_executor = create_json_agent(
    llm=chosen_llm,
    toolkit=json_toolkit,
    verbose=True
)

# Run the agent
input_text = "Wat is de _mean waarde van bu_naam De Slank?"
response = json_agent_executor.invoke(input_text, handleParsingErrors=True)
print (response)



> Entering new AgentExecutor chain...
Action: json_spec_list_keys  
Action Input: data  
Observation: ['type', 'features', 'totalFeatures', 'numberMatched', 'numberReturned', 'timeStamp', 'crs']
Thought:The keys do not seem to directly explain what I am looking for regarding "_mean waarde van bu_naam De Slank". I should explore the "features" key as it might contain individual entries that have more information. 

Action: json_spec_list_keys  
Action Input: data["features"]  
Observation: ValueError('Value at path `data["features"]` is not a dict, get the value directly.')
Thought:The "features" key does not contain a dictionary but rather a list of items. I need to access the first feature to see if it contains information regarding "bu_naam De Slank".

Action: json_spec_list_keys  
Action Input: data["features"][0]  
Observation: ValueError('Value at path `data["features"][0]` is not a dict, get the value directly.')
Thought:The first feature seems to point to a more complex struct